In [24]:
import tensorflow as tf
import numpy as np
import csv
import glob
from PIL import Image

In [25]:
yes_folder = 'Note_ja_nein_mama/ja'
no_folder = 'Note_ja_nein_mama/nein'
with open('X_notes.csv', 'w') as Xf:
    Xwriter = csv.writer(Xf)
    with open('Y_notes.csv','w') as Yf:
        Ywriter = csv.writer(Yf)
        for note_file in sorted(glob.glob(yes_folder + '/*.jpg')):
            note = Image.open(note_file)
            note = np.asarray(note)
            to_save = np.reshape(note,(1,note.shape[0]*note.shape[1]))
            Xwriter.writerows(to_save)
            Y = np.reshape(np.array(1),(1,1))   
            Ywriter.writerows(Y)
        for note_file in sorted(glob.glob(no_folder + '/*.jpg')):
            note = Image.open(note_file)
            note = np.asarray(note)
            to_save = np.reshape(note,(1,note.shape[0]*note.shape[1]))
            Xwriter.writerows(to_save)
            Y = np.reshape(np.array(0),(1,1))   
            Ywriter.writerows(Y)

In [26]:
train_size = 0.8
X = np.genfromtxt('X_notes.csv', delimiter=',')
Y = np.genfromtxt('Y_notes.csv', delimiter=',')

m = X.shape[0]
size = X.shape[1]
rand_index = list(np.random.permutation(m))
m_train = int(m*train_size)
X_train = X[rand_index[:m_train],:]
Y_train = np.reshape(Y[rand_index[:m_train]],(m_train,1))
Y_train = np.append(Y_train, 1-Y_train, axis=1)
X_test = X[rand_index[m_train:],:]
Y_test = np.reshape(Y[rand_index[m_train:]],(m-m_train,1))
Y_test = np.append(Y_test, 1-Y_test, axis=1)

In [28]:
x = tf.placeholder(tf.float32, [None, size])
W = tf.Variable(tf.zeros([size, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
sess.run(train_step, feed_dict={x: X_train, y_: Y_train})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: X_test, y_: Y_test}))

0.67474


In [29]:
print(sess.run(y, feed_dict={x: X_test}))

[[ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 0.94158542  0.05841456]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 

In [30]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [31]:
W_conv1 = weight_variable([5, 5, 1, 20])
b_conv1 = bias_variable([20])
x = tf.placeholder(tf.float32, [None, size])
y_ = tf.placeholder(tf.float32, [None, 2])
x_image = tf.reshape(x, [-1,30,80,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 20, 20])
b_conv2 = bias_variable([20])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([6 * 12 * 20, 20])
b_fc1 = bias_variable([20])

h_pool2_flat = tf.reshape(h_pool2, [-1, 6 * 12 * 20])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([20, 2])
b_fc2 = bias_variable([2])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [32]:
sess.run(tf.initialize_all_variables())
train_step.run(session = sess, feed_dict={x: X_train, y_: Y_train, keep_prob: 0.5})
print("test accuracy %g"%accuracy.eval(session = sess, feed_dict={x: X_test, y_: Y_test, keep_prob: 1.0}))

InvalidArgumentError: Incompatible shapes: [1152,2] vs. [2560,2]
	 [[Node: mul_2 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_Placeholder_5_0, Log_2)]]
Caused by op u'mul_2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/IPython/config/application.py", line 574, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/IPython/kernel/zmq/kernelapp.py", line 373, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 151, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/ioloop.py", lin

In [32]:
print(sess.run(y, feed_dict={x: X_test}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_10' with dtype float
	 [[Node: Placeholder_10 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_10', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/IPython/config/application.py", line 574, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/IPython/kernel/zmq/kernelapp.py", line 373, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 151, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib

In [41]:
sess.close()